# Modules

We saw in the chapter on _Subroutines_ that it is possible to break a program into re-usable self-contained units. However, when they are included in the same file as the main program they are not truly independent. We could create a subroutine to perform a complex calculation, for example, but it would still have to be compiled alongside a potentially unrelated function to do some plotting. _Modules_ solve this problem by allowing units to be packaged, tested, compiled and distributed separately, allowing the creation of a toolbox of useful functions. For those with experience of other programming language, Fortran modules resemble _Python_ modules but with features of Python and C++ _Classes_. 

## Inside a Module

A _Module_ can contain variable declarations, which have scope throughout the module, and also units such as subroutines, which can _also_ contain declarations with restricted scope. It cannot contain _bare_ executable statements. The best way to demonstrate what a module looks like is with a simple example:

```fortran
Module my_module
  Implicit none
  ! Ensure that all declared items have scope restricted to this module only
  ! unless stated otherwise.
  Private
  
  ! Allow the 'add_vars' subroutine to be accessed from outside (public).
  Public   :: add_vars   
  
  ! These values are accessible within the module but not from outside (private).
  Integer  :: my_counter = 0
  
  ! A module can 'contain' multiple units, which can either be public or private.
  contains
    integer function add_vars(a, b)
      Implicit none
      
      ! Dummy arguments
      Integer, intent(in)  :: a
      Integer, intent(in)  :: b
      
      ! Increment the access counter and print it out
      my_counter = my_counter + 1
      print *, 'Number of times function called: ', my_counter
      
      ! Add two values and return the result
      c = a + b

    end function
end module my_module
```

A module always begins with `Module`, and ends with `End module`. Within in we can place declaration statements like `Implicit none` but not program code. This is why the variable `my_counter` has to be declared and initialised in the same step. The first statement, `Private`, is recommended at the beginning of a module as it forces all subsequent contents to have scope restricted to that module unless made explicitly `Public`.

The `contains` statement signifies the point of the module where declarations end and the payload of program units begins. In this case we have a single function that adds two values and returns the result, very similar to the example we saw earlier. We have made it public so that it can be accessed from the main program, otherwise it wouldn't be very useful!

The module can be accessed by another program unit by using the `use` statement, which must always come before the `implicit none` statement. This is the simplest example:

```fortran
program moduletest
  use my_module
  Implicit none
end program
```

A fully working example can be found in the following location:

| Program        | Directory              | Purpose                                               |
| -------------- | ---------------------- | ----------------------------------------------------- |
| **moduletest** | `src/modules/example1` | Call a function defined in a module.                  |

This program calls our `add_vars()` function with a range of values. Whilst extremely simple it demonstrates a potentially very useful features of modules, _persistence_. The module itself remains in scope the whole time the main program is running, and because the `my_counter` variable belongs to the scope of the module, not the subroutine, its value persists between function calls. This allows a Fortran module to mimic the behaviour of a `Class`, in that it can encapsulate both data and functionality in one package.

Whilst it is possible to call a bare subroutine from a different file that hasn't been wrapped in a module, this will lead to extra complications in that we will need to define an explicit _interface_ for the subroutine so that the main program knows what number and type of arguments the subroutine is expecting. Using modules means interfaces are generated automatically and we don't need to worry about them.

### Exercise 1 - _Modules_

 * The variable `my_counter` is declared in the module alongside the function `add_vars()`. Can you access it from the main program. If not, why not? Can you modify the code so that it can be accessed?
 * Remove the initialisation of the variable `my_counter` (setting it to zero). Do the results change? Do you trust them?
 * When large numbers of module variables need to be initialised, potentially with values read from a file or passed via user input, it is often neater to create a separate function to do this. Create another function called `init()` or similar, that sets `my_counter` to zero and modify the main program so that this function is called before `add_vars()` is ever used. Try setting the initial value to something other than zero.

## Common Blocks

Those familiar with Fortran 77 may use common blocks to transfer persistent values between program units. This is not recommended as it makes debugging more difficult. Without defined entry and exit points to subroutines and functions it can be hard to determine where bad values can be set and renders those program units dependent upon external code. They can no longer be self-contained. Modules can be used to replicate the concept of persistent values across program units, but using them as exact duplicates is discouraged as it simply replicates the problems. Rewriting the code to take full advantage of modules will reduce problems of testing and extensibility in future.

# Array Operations

Efficient use of arrays is of crucial importance in time critical, iterative operations. The burden on the software developer is to remember how Fortran stores arrays (in column-major order) and in which circumstances Fortran will be forced to make a copy of an array, or part of an array, thus taking time and additional memory. Although an array may be two dimensional it is stored as a linear series of values, column by column. Accessing contiguous memory locations (one following the next) is always faster the jumping around, so trying to ensure that your code works in this manner will ensure it is running as fast as possible. Some examples of good practice are shown below. 

## Looping

Remember the example of a two dimensional array discussed in the _Arrays_ section. An array that looks like this:

|[]()|[]()|
|----|----|
| 1  | 2  |
| 3  | 4  |
| 5  | 6  |

...is stored in memory like this:

|[]()|[]()|[]()|[]()|[]()|[]()|
|----|----|----|----|----|----|
| 1  | 3  | 5  | 2  | 4  | 6  |

In order access memory in a contiguous manner, we would need to span the data column by column, rather than row by row. 

In [ ]:
%num_images: 1
Program array_loop
  Implicit none
  ! Declare a two dimentsional array
  Integer  :: arrName(3, 2)
  
  ! Declare loop counters
  Integer  :: x, y
  
  ! Assign some values
  arrName = reshape([1, 3, 5, 2, 4, 6], shape(arrName))
  
  ! Loop over arrays and print out values
  do y = 1, 2    ! 
    do x = 1, 3  ! Loop over column slice of contiguous memory
      print *, 'value = ', arrName(x, y)
    end do
  end do

End program

Looping over such a small array takes too short a time to show a real difference but, you can find larger examples using a `(50000, 4000)` array in the following directory:

| Program         | Directory             | Purpose                                                    |
| --------------- | --------------------- | ---------------------------------------------------------- |
| **efficient**   | `src/arrays/example1` | Loop over a two dimensional array in an efficient manner   |
| **inefficient** | `src/arrays/example1` | Loop over a two dimensional array in an inefficient manner |

### Exercise 2 - _Looping_

 * Run these two examples. How much longer does the inefficient version take?
 * Bearing in mind how two-dimensional arrays are stored, what would you expect to happen to the ratio of the speeds of the two methods if the number of columns were increased relative to the number of rows for an array with the same number of elements? Try it and see.

## Slicing

## Passing as Arguments